In [16]:
import xarray as xr
import numpy as np
import skimage.measure
import pandas as pd
from scipy.spatial import distance
import skimage.measure
from skimage.morphology import remove_small_objects,closing,binary_closing
from scipy import ndimage
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from datetime import datetime,timedelta
import os
from dask.distributed import Client
import glob
from mrms_gage import *

In [17]:
# Create a path to the code file
codeDir = os.path.dirname(os.path.abspath(os.getcwd()))
parentDir = os.path.dirname(codeDir)

In [18]:
temp_folder = os.path.join(parentDir,"predict_temp")


mrms_folder = os.path.join(parentDir,"2min_rate_cat_month_CO","eval")
os.chdir(mrms_folder)
filenames_rate = glob.glob('*.grib2')

In [22]:
# multisensor correction
# open multisensor qpe for 2021,2022
mrms_multi_folder = os.path.join(parentDir,"1hr_QPE_multi_cat_yr_CO")
filenames_multi = glob.glob(mrms_multi_folder+'\\'+'*.grib2')
mrms_multi = xr.open_mfdataset(filenames_multi,engine = "cfgrib",chunks={'time': '1MB'})
mrms_multi = mrms_multi.where(mrms_multi>=0)
mrms_multi = mrms_multi.where(mrms_multi.longitude<=255,drop=True)

# open radaronly qpe for 2021,2022
mrms_radar_folder = os.path.join(parentDir,"1hr_QPE_radar_cat_yr_CO")
filenames_radar = glob.glob(mrms_radar_folder+'\\'+'*.grib2')
mrms_radar = xr.open_mfdataset(filenames_radar,engine = "cfgrib",chunks={'time': '1MB'})
mrms_radar = mrms_radar.where(mrms_radar>=0)
mrms_radar = mrms_radar.where(mrms_radar.longitude<=255,drop=True)

correction = (mrms_multi/mrms_radar)
correction = correction.where(correction.unknown != np.inf).fillna(1)

Ignoring index file 'Z:\\MRMS\\1hr_QPE_multi_cat_yr_CO\\2022_multiQPE_CO.grib2.923a8.idx' incompatible with GRIB file
C:\Users\whitep\Miniconda3\envs\radar\lib\site-packages\xarray\core\indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
C:\Users\whitep\Miniconda3\envs\radar\lib\site-packages\xarray\core\indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config

In [5]:
for i in range(len(filenames_rate)):
    mrms_rate = xr.open_dataset(filenames_rate[i],engine = "cfgrib",chunks={'time': '50MB'})
    mrms_rate = mrms_rate.where(mrms_rate.longitude<=255,drop=True)
    
    mrms_accum = mrms_rate*(2/60)
    # multisensor correction
    correction_month = correction.sel(time=slice(mrms_accum.time[0],mrms_accum.time[-1]))
    correction_month = correction_month.resample(time='2min').pad()
    
    mrms_2_corrected = mrms_accum*correction_month
    # change time to MST
    mrms_2_corrected = time_change(mrms_2_corrected)
    # get rid of neg values
    mrms_2_corrected = mrms_2_corrected.where(mrms_2_corrected>=0)

    mrms_one = mrms_2_corrected.resample(time='1min').asfreq()
    mrms_one = mrms_one.unknown.fillna(0)
    mrms_15 = (mrms_one.rolling(time=15,min_periods=1).sum())*(60/15) 
    
    name = filenames_rate[i][0:8]+'_15int'
    path = 'Z:\\predict_temp\\'+name+'.nc'
    mrms_15.to_netcdf(path=path)

Can't read index file 'aug_2021_rate_CO.grib2.923a8.idx'
Traceback (most recent call last):
  File "C:\Users\whitep\Miniconda3\envs\radar\lib\site-packages\cfgrib\messages.py", line 547, in from_indexpath_or_filestream
    self = cls.from_indexpath(indexpath)
  File "C:\Users\whitep\Miniconda3\envs\radar\lib\site-packages\cfgrib\messages.py", line 429, in from_indexpath
    index = pickle.load(file)
EOFError: Ran out of input
C:\Users\whitep\Miniconda3\envs\radar\lib\site-packages\xarray\core\indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
C:\Users\whitep\Miniconda3\envs\radar\lib\site-packages\dask\core.py:119: Run